In [1]:
%load_ext autoreload
%autoreload 2

### **IMPORTING LIBRARY AND SETTING PATH**

In [7]:
# Needed Libraries
import numpy as np
from scipy import signal as sig, stats
from Signal_Analysis_Class import heart_rate
import wfdb as wf
import pywt as WT
import matplotlib.pyplot as plt
import os
from multiprocessing.pool import Pool
from pydmd import HODMD
import warnings



# Specify the default path
os.chdir('..')
dir = os.getcwd()

# Global variables
results = []

### **FUNCTION DEFINITIONS**

In [3]:
def Signal_Filter(Signal):
    # Wavelet Tranformation
    Coeff = WT.wavedec(Signal, 'db3', level = 11)
    Coeff_Mod = []

    for i in range(0,11):
        if i > 3:
            Coeff_Mod.append(Coeff[i])
        else:
            Coeff_Mod.append(np.zeros(Coeff[i].shape))

    # Inverse Wavelet Transform
    Signal_WT = WT.waverec(Coeff_Mod, 'db3')

    # Notch Filter
    samp_freq = 1000
    notch_freq = 60.0
    quality_factor = 2

    b_notch, a_notch = sig.iirnotch(notch_freq, quality_factor, samp_freq)
    Signal_Notch =  sig.filtfilt(b_notch, a_notch, Signal_WT)

    # Fourth order Butterworth low pass filter at 100Hz
    sos = sig.butter(N = 4, Wn = 100,output ='sos', fs=1000)
    return sig.sosfilt(sos, Signal_Notch)

def Patient_Signals_Filter(Signals_Array):
    Signals_Filtered_Array = None

    for s in Signals_Array:
        Signal_Filtered = Signal_Filter(s)

        if str(type(Signals_Filtered_Array)) == "<class 'NoneType'>":
            Signals_Filtered_Array = Signal_Filtered
        else:
            Signals_Filtered_Array = np.vstack((Signals_Filtered_Array,Signal_Filtered))

    return Signals_Filtered_Array

def Beat_ind(Signal_Filterd_Array):

    Peak_ind = None

    for s in Signal_Filterd_Array:

        hr = heart_rate(s,1000)
        result = np.array(hr.find_r_peaks())

        result = np.unique(result[result > 0])
        result = result[1:len(result)-1]

        for i in range(1,len(result)-1):
            if result[i] > 300:
                valid_peaks = [result[i]]
                break

        for i in range(1,len(result)-1):
            if result[i] - valid_peaks[-1] > 250:
                valid_peaks.append(result[i])
        
        valid_peaks = np.array(valid_peaks)
        result = valid_peaks

        if str(type(Peak_ind)) == "<class 'NoneType'>":
                Peak_ind = np.array([result])
        else:
            if Peak_ind.shape[1] == result.shape[0]:
                Peak_ind = np.vstack((Peak_ind,result))
            elif Peak_ind.shape[1] > result.shape[0]:
                Peak_ind = np.vstack((Peak_ind[:,:result.shape[0]],result))
            elif Peak_ind.shape[1] < result.shape[0]:
                Peak_ind = np.vstack((Peak_ind,result[:Peak_ind.shape[1]]))
            else:
                print("Indexing error!!!")
                break
    
    return Peak_ind

def Beat_Signal_Extraction(Ori_signal, ind):

    Sample_Signal = None
    for n in range(12):
        if str(type(Sample_Signal)) == "<class 'NoneType'>":
            Sample_Signal = Ori_signal[n,int(ind[n]-250):int(ind[n]+400)]
        else:
            Sample_Signal = np.vstack((Sample_Signal,Ori_signal[n,int(ind[n]-250):int(ind[n]+400)]))
    return Sample_Signal

def Arg_X(Sample_signal):
    
    B = Sample_signal.flatten(order = 'F')
    N = (Sample_signal.shape[1] - 109) + 1

    Arg_Matrix = None

    for n in range(N):
        V = B[n*12:n*12 + 109*12]
        if str(type(Arg_Matrix)) == "<class 'NoneType'>":
            Arg_Matrix = V
        else:
            Arg_Matrix = np.vstack((Arg_Matrix,V))
    
    return Arg_Matrix.T

def FeatureExtraction_Beat(Signal_F, ind,n):

    A = Beat_Signal_Extraction(Signal_F, ind)
    B = Arg_X(A)

    X_DMD =  HODMD(svd_rank = 0, exact= True)
    X_DMD.fit(B)

    Lambda = X_DMD.eigs
    Modes = X_DMD.modes    

    M = int(Modes.shape[0]/12)

    Modes_restacked = np.zeros((12,Modes.shape[1]))

    for j in range(M):
        Modes_restacked = Modes_restacked + Modes[int(j*12):int(j*12 + 12),:]

    Modes_restacked = Modes_restacked/M

    Mean = np.mean(Modes_restacked, axis = 1)
    Std = np.std(Modes_restacked, axis = 1)
    Var = np.var(Modes_restacked, axis = 1)
    skew = stats.skew(Modes_restacked, axis = 1)
    kur = stats.kurtosis(Modes_restacked, axis = 1)

    Feature_String = Numpy_arr_to_String(Mean) + Numpy_arr_to_String(Std) + Numpy_arr_to_String(Var) + Numpy_arr_to_String(skew) + Numpy_arr_to_String(kur)
    return (str(n) + ',' + Feature_String)

def Numpy_arr_to_String(Arr):
    string = ''
    for no in Arr:
        string = string + ',' + str(no) 

    return string

def get_results(result):
    global results
    results.append(result)

def Patient_FeatureExtraction_Beat(patient, name):
    Signal_Array, fields = wf.rdsamp( dir + '/DATASET/' + patient + '/' + name)

    Signal_Array = Signal_Array.T
    Signal_Array = Signal_Array[:12,:]

    Signal_Filterd_Array = Patient_Signals_Filter(Signal_Array)

    Beat_index = Beat_ind(Signal_Filterd_Array)

    Peaks_count = Beat_index.shape[1]

    Result_File = []
    print(f"Patient: {patient} Audio ID: {name}")
    for n in range(Peaks_count):
        Result_File.append(patient + '_' + FeatureExtraction_Beat(Signal_Filterd_Array,Beat_index[:,n],n))
        
    return(Result_File)

### **LOADING DATASET**

In [4]:
# Specifying Dataset Directory
os.chdir(dir + '/DATASET')
Patients_List = os.listdir()
os.chdir(dir)

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    print(fields['comments'][4])


# Healthy control List
HC = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Healthy control' in fields['comments'][4]:
        HC.append([patient,F[0].replace('.dat','')])

# Myocardial infarction
MI = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocardial infarction' in fields['comments'][4]:
        MI.append([patient,F[0].replace('.dat','')])

# Cardiomyopathy/Heart failure
HF = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Cardiomyopathy' in fields['comments'][4] or 'Heart failure' in fields['comments'][4]:
        HF.append([patient,F[0].replace('.dat','')])

# Bundle branch block
BBB = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Bundle branch block' in fields['comments'][4]:
        BBB.append([patient,F[0].replace('.dat','')])

# Dysrhythmia
DA = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Dysrhythmia' in fields['comments'][4]:
        DA.append([patient,F[0].replace('.dat','')])

# Myocardial hypertrophy
MH = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocardial hypertrophy' in fields['comments'][4]:
        MH.append([patient,F[0].replace('.dat','')])

# Valvular heart disease
VH = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Valvular heart' in fields['comments'][4]:
        VH.append([patient,F[0].replace('.dat','')])

# Myocarditis
MC = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocarditis' in fields['comments'][4]:
        MC.append([patient,F[0].replace('.dat','')])


Reason for admission: Heart failure (NYHA 2)
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Dysrhythmia
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: n/a
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Dysrhythmia
Reason for admission: Myocardial infarction
Reason for admission: n/a
Reason for admission: Myocardial infarcti

### **SEGEMENTATION**

#### **BEAT WISE** - **Myocarditis**

In [5]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MC

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Myocarditis.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient269 Audio ID: s0508_re
Patient: patient271 Audio ID: s0509_re
Patient: patient249 Audio ID: s0484_re
Patient: patient272 Audio ID: s0510_re


AttributeError: module 'numpy' has no attribute 'skew'

#### **BEAT WISE** - **Valvular heart disease**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = VH

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Valvular heart disease.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Myocardial hypertrophy**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MH

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat__Myocardial hypertrophy.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Dysrhythmia**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = DA

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/Basic/Features_Beat_Dysrhythmia.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Bundle branch block**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = BBB

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/Basic/Features_Beat_Bundle branch block.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Cardiomyopathy/Heart failure**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = HF

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/Basic/Features_Beat_NoArg_Cardiomyopathy_HeartFailure.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Myocardial infarction**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MI[:60]
print(len(Z))

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/Basic/Features_Beat_Myocardial infarction.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Health Control**

In [ ]:
Z = HC
print(len(Z))

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/Basic/Features_Beat_Health Control.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()